# 3. Text preprocessing

## Setup

In [1]:
%run __init__.py

INFO:root:Starting logger


In [2]:
import pandas as pd

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Common pipeline

## COVID-19
bla bla bla

### Loading the dataframe

In [ ]:
CORD_DATASET_DIR = os.path.join(DATA_DIR, 'cord19')
CORD19_FILE_PATH = os.path.join(CORD_DATASET_DIR, 'cord19_dataframe.pkl')

cord19_df = pd.read_pickle(CORD19_FILE_PATH)

## Agriculture

### Loading the dataframe

In [3]:
AGRICULTURE_DATASET_DIR = os.path.join(DATA_DIR, 'agriculture')
PMC_FILE_PATH = os.path.join(AGRICULTURE_DATASET_DIR, 'pmc_dataframe.pkl')

pmc_df = pd.read_pickle(PMC_FILE_PATH)

### Preprocessing text

In [4]:
publications = pmc_df['text_cleaned'].values

In [49]:
import numpy as np
import multiprocessing as mp

import string
import spacy 
import en_core_web_sm
from sklearn.base import TransformerMixin, BaseEstimator

nlp = en_core_web_sm.load()
nlp.Defaults.stop_words |= {"et","al", "introduction", "Fig"}

class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, n_jobs=1):
        self.n_jobs = n_jobs

    def fit(self, X, y=None):
        return self

    def transform(self, X, *args, **kwargs):
        return [self._preprocess_text(text) for text in X]

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        doc = nlp(text)
        return [t.lemma_ for t in doc if len(t.text) > 2 and
                not t.is_stop and t.text not in string.punctuation
                and t.is_alpha and not t.is_digit]

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if not t.is_stop]
    
    def _remove_digits(self, doc):
        return [t for t in doc if t.is_alpha and not t.is_digit]

    def _lemmatize(self, doc):
        return [t.lemma_ for t in doc if len(t.text) > 2]
    

### LDA

In [65]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

def dummy(doc):
    return doc

lda_pipeline = Pipeline([('preprocessing', TextPreprocessor()),
                     ('vectorizer', CountVectorizer(preprocessor=dummy, tokenizer=dummy)),
                     ('model', LatentDirichletAllocation())])


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

search_params = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'model__n_components': [10, 15, 20, 25, 30],
    'model__learning_decay': [.5, .7, .9]
}

search = RandomizedSearchCV(lda_pipeline, param_distributions=search_params, n_iter=15)
search.fit(publications)

In [ ]:
best_lda_model = search.best_estimator_

print(f"Best pipeline parameters: {search.best_params_}")
print(f"Best Log Likelihood Score: {search.best_score_}")

In [52]:
tf_feature_names = pipeline['vectorizer'].get_feature_names()
print_top_words(pipeline['model'], tf_feature_names, 10)

Topic #0: plant transgenic gene line base control datum show sensor node
Topic #1: plant soil root carbon cell different phytolith sample non high
Topic #2: plant yield high temperature water crop grain leaf stress result
Topic #3: system crop food farmer farm level practice change study increase
Topic #4: plant infect protein infection infected Las non root virus shoot
Topic #5: plant stress increase level high gene growth expression treatment condition
Topic #6: rice soil root crop Fig treatment high water increase sample
Topic #7: plant gene protein expression stress figure show mutant response Fig
Topic #8: habitat farming mussel mission grassland agent target cost value bird
Topic #9: plant medicinal specie take informant Herb decoction virus orally report



In [9]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()



Visualization. Do this with the best model obtained before.

In [53]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

In [62]:
preprocessed_text = TextPreprocessor().fit_transform(publications)
tf_vectorizer = CountVectorizer(preprocessor=dummy, tokenizer=dummy, ngram_range=(1, 3))
dtm_tf = tf_vectorizer.fit_transform(preprocessed_text)
lda_tf = LatentDirichletAllocation().fit(dtm_tf)

In [63]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
9      32.420952   8.115230       1        1  26.400705
1       0.648955  71.707100       2        1  18.654967
6      71.019684   9.971006       3        1  12.538139
2       3.785981  33.185135       4        1  11.882039
0     -42.666500  -9.222070       5        1   9.890597
4      -3.621689 -43.526516       6        1   5.938765
3      -3.632402  -4.171708       7        1   5.672064
5     -36.584400  35.804413       8        1   3.907036
8      44.391380  50.615383       9        1   3.812686
7      40.235809 -31.591665      10        1   1.303002, topic_info=             Term          Freq         Total Category  logprob  loglift
514159      plant  10540.000000  10540.000000  Default  30.0000  30.0000
645228       soil   1667.000000   1667.000000  Default  29.0000  29.0000
685865     system   1017.000000   1017.000000  Default  28.0000  28.0000
602765       rice   1368.000000   1368.000000  Default  27.0000  27.0000
324197       food    483.000000    483.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
419851      level     14.556681   1583.323058  Topic10  -7.2131  -0.3487
670675  structure      9.157627    204.869226  Topic10  -7.6766   1.2327
644470     social      7.496379     84.655264  Topic10  -7.8767   1.9163
293744    example      8.327003    255.983436  Topic10  -7.7716   0.9149
530563     policy      6.873412     79.988386  Topic10  -7.9635   1.8863

[767 rows x 6 columns], token_table=        Topic      Freq       Term
term                              
851         3  0.994382       ALDH
867         3  0.968463  ALDH gene
1114        9  0.997686        AMV
1124        9  0.946073    AMV CMV
1379        8  0.986510       APRI
...       ...       ...        ...
751272      8  0.123445      yield
751272      9  0.017443      yield
753062      1  0.081361       zinc
753062      4  0.904015       zinc
753062      5  0.009040       zinc

[2018 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 7, 3, 1, 5, 4, 6, 9, 8])

### Latent Semantic Analysis (LSA)

In [ ]:
from sklearn.decomposition import TruncatedSVD

lsa_pipeline = ([('preprocessing', TextPreprocessor()),
                 ('vectorizer', TFIDFVectorizer()),
                 ('model', TruncatedSVD())])

### NMF

In [ ]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TFIDFVectorizer

pipeline = Pipeline([('preprocessing', TextPreprocessor()),
                     ('vectorize', TFIDFVectorizer()),
                     ('model', NMF())])